### Instalacja bibliotek

In [ ]:
pip install tensorflow

### Importowanie potrzebnych bibliotek

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

### Definicja modelu

In [ ]:
model = Sequential([
    # Pierwsza warstwa konwolucyjna
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    # Druga warstwa konwolucyjna
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Trzecia warstwa konwolucyjna
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Spłaszczenie wyników do jednowymiarowego wektora
    Flatten(),
    # Warstwa gęsta do klasyfikacji
    Dense(512, activation='relu'),
    # Warstwa wyjściowa
    Dense(1, activation='sigmoid')
])

### Kompilacja modelu

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Podsumowanie modelu

In [ ]:
model.summary()